# Azure AI Agent Framework SDK 테스트 (Foundry)

이 노트북은 Azure AI Projects SDK를 사용하여 배포된 `gpt-4o-mini` 모델과 상호 작용하는 방법을 보여줍니다.

## 사전 요구 사항
1. `azd up`을 사용하여 인프라를 배포합니다.
2. `azd env get-values > src/.env`를 실행하여 환경 변수 파일을 생성합니다.
3. Python 가상 환경(.venv)이 구성되어 있고, **이 노트북의 커널로 '.venv'가 선택**되어 있는지 확인하세요.

In [ ]:
%pip install azure-ai-projects azure-identity python-dotenv openai

In [ ]:
import os
import time
import json
import traceback
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
# azure-ai-projects SDK v1.0.0+에서는 도구 정의 클래스들이 azure.ai.agents.models에 위치합니다.
try:
    from azure.ai.projects.models import (
        ToolDefinition,
        CodeInterpreterToolDefinition,
        FunctionToolDefinition,
        FileSearchToolDefinition,
        FunctionDefinition
    )
except ImportError:
    from azure.ai.agents.models import (
        ToolDefinition,
        CodeInterpreterToolDefinition,
        FunctionToolDefinition,
        FileSearchToolDefinition,
        FunctionDefinition
    )

from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

project_connection_string = os.getenv("AZURE_AI_PROJECT_CONNECTION_STRING")
deployment_name = "gpt-4o-mini" # 배포된 모델 이름 (infra/main.bicep에서 설정한 이름)

print(f"Project Connection String: {project_connection_string}")
print(f"Deployment Name: {deployment_name}")

In [ ]:
# 설정
project_connection_string = os.getenv("AZURE_AI_PROJECT_CONNECTION_STRING")

if not project_connection_string:
    raise ValueError("AZURE_AI_PROJECT_CONNECTION_STRING이 설정되지 않았습니다. 'azd env get-values > src/.env'를 실행하세요.")

print(f"프로젝트에 연결 중: {project_connection_string}")

In [ ]:
# 클라이언트 초기화
credential = DefaultAzureCredential()
client = AIProjectClient(
    endpoint=project_connection_string,
    credential=credential
)

In [ ]:
# 에이전트 생성 (또는 간단한 채팅 완료)
# 참고: 프로젝트 클라이언트에서 OpenAI 클라이언트를 가져와서 사용

deployment_name = "gpt-4o-mini"

# client.inference 속성이 없으므로 client.get_openai_client()를 직접 호출합니다.
# api_version은 필수로 지정해야 합니다.
openai_client = client.get_openai_client(api_version="2024-06-01")

try:
    response = openai_client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "당신은 도움이 되는 어시스턴트입니다."},
            {"role": "user", "content": "Python 프로그래밍에 대한 농담을 하나 해주세요."}
        ]
    )

    print("응답:")
    print(response.choices[0].message.content)
finally:
    openai_client.close()

In [ ]:
# 에이전트 프레임워크 테스트 (미리보기)
# 배포된 지역/버전에서 'azure-ai-projects' 특정 에이전트 기능을 사용할 수 있어야 합니다.
import traceback
import time

try:
    # 1. 에이전트 생성
    agent = client.agents.create_agent(
        model=deployment_name,
        name="python-tutor",
        instructions="당신은 Python 튜터입니다."
    )
    
    print(f"에이전트 생성됨: {agent.id}")
    
    # 2. 스레드 생성
    thread = client.agents.threads.create()
    print(f"스레드 생성됨: {thread.id}")
    
    # 3. 메시지 추가
    message = client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content="리스트 컴프리헨션(list comprehensions)에 대해 설명해 주세요."
    )
    
    # 4. 에이전트 실행
    # 주의: runs.create는 'assistant_id' 대신 'agent_id' 매개변수를 사용합니다.
    run = client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)
    
    # 실행 상태 폴링
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        # 실행 상태 업데이트
        run = client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        print(f"실행 상태: {run.status}")
        
    if run.status == "failed":
        print(f"실행 실패: {run.last_error}")
    else:
        # 5. 응답 메시지 조회
        messages = client.agents.messages.list(thread_id=thread.id)
        print("에이전트 응답:")
        
        # messages는 Iterator(ItemPaged)이므로 리스트로 변환하여 접근합니다.
        messages_list = list(messages)
        
        # 첫 번째 메시지 (보통 가장 최근 메시지)
        if messages_list:
             print(messages_list[0].content[0].text.value)
        else:
             print("메시지가 없습니다.")

    # 6. 에이전트 정리 (삭제)
    client.agents.delete_agent(agent.id)
    print("에이전트 삭제됨")
    
except Exception:
    traceback.print_exc()

In [ ]:
# [고급 예제] 코드 인터프리터(Code Interpreter) 도구 사용
# 에이전트가 Python 코드를 실행하여 복잡한 계산이나 데이터 처리를 수행하도록 합니다.
# (참고: Azure 지역에 따라 Code Interpreter 지원 여부가 다를 수 있습니다.)

try:
    print("--- 코드 인터프리터 테스트 시작 ---")
    
    # 1. Code Interpreter 도구 정의
    # 참고: SDK 버전에 따라 ToolDefinition 또는 CodeInterpreterTool 정의 방식이 다를 수 있습니다.
    # azure-ai-agents 1.0.0+ 에서는 CodeInterpreterToolDefinition()을 사용합니다.
    code_interpreter = CodeInterpreterToolDefinition()

    # 2. 에이전트 생성 (도구 포함)
    math_agent = client.agents.create_agent(
        model=deployment_name,
        name="math-tutor",
        instructions="당신은 수학 튜터입니다. 복잡한 계산은 코드를 작성하고 실행하여 해결하세요.",
        tools=[code_interpreter]
    )
    print(f"수학 에이전트 생성됨: {math_agent.id}")

    # 3. 스레드 생성
    thread = client.agents.threads.create()
    
    # 4. 피보나치 수열 계산 요청
    client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content="피보나치 수열의 10번째 항을 계산해 주세요. 코드를 실행해서 확인해 주세요."
    )
    
    # 5. 실행
    run = client.agents.runs.create(thread_id=thread.id, agent_id=math_agent.id)
    
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        print(f"실행 상태: {run.status}")

    if run.status == "failed":
        print(f"실행 실패: {run.last_error}")
    else:
        messages = client.agents.messages.list(thread_id=thread.id)
        messages_list = list(messages)
        # 최신 메시지 출력 (에이전트의 답변)
        if messages_list:
            response_content = messages_list[0].content[0].text.value
            print(f"에이전트 응답: {response_content}")
            
            # 코드 실행 로그나 출력이 있는지 확인 (MessageContent에 포함될 수 있음)
            # SDK 구조에 따라 image_file 등이 반환될 수도 있습니다.
            
    # 에이전트 정리
    client.agents.delete_agent(math_agent.id)
    print("수학 에이전트 삭제됨")
    
except Exception as e:
    print(f"코드 인터프리터 테스트 실패 (지역 지원 여부 확인 필요): {e}")
    # 실패 시 생성된 에이전트가 있다면 정리 시도
    try:
        if 'math_agent' in locals():
            client.agents.delete_agent(math_agent.id)
    except:
        pass

In [ ]:
# [고급 예제] 함수 호출 (Function Calling) 도구 사용
# 에이전트가 외부 함수(예: 날씨 정보 조회)를 호출해야 할 때를 판단하고 인자를 생성합니다.

try:
    print("--- 함수 호출(Function Calling) 테스트 시작 ---")

    # 1. 사용자 정의 함수 정의 (Mock)
    def get_weather(location):
        # 실제로는 외부 API를 호출하겠지만, 여기서는 Mock 데이터를 반환합니다.
        if "서울" in location:
            return "서울의 날씨는 맑음, 기온 20도입니다."
        else:
            return f"{location}의 날씨는 흐림, 기온 15도입니다."

    # 2. 도구 정의 (Function Tool)
    # ToolDefinition 대신 FunctionToolDefinition과 FunctionDefinition 사용 (v1.x SDK)
    weather_tool = FunctionToolDefinition(
        function=FunctionDefinition(
            name="get_weather",
            description="특정 위치의 현재 날씨를 가져옵니다.",
            parameters={
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시 또는 지역 이름 (예: 서울, 부산)"
                    }
                },
                "required": ["location"]
            }
        )
    )

    # 3. 에이전트 생성 (함수 도구 포함)
    weather_agent = client.agents.create_agent(
        model=deployment_name,
        name="weather-bot",
        instructions="당신은 날씨 안내원입니다. 사용자가 날씨를 물어보면 도구를 사용하여 답변하세요.",
        tools=[weather_tool]
    )
    print(f"날씨 에이전트 생성됨: {weather_agent.id}")

    # 4. 스레드 생성 및 메시지 추가
    thread = client.agents.threads.create()
    client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content="서울 날씨 어때?"
    )

    # 5. 실행
    run = client.agents.runs.create(thread_id=thread.id, agent_id=weather_agent.id)

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        print(f"실행 상태: {run.status}")

        # [중요] 함수 호출 요구 시 처리 로직
        if run.status == "requires_action":
            print(" -> 함수 실행 요구됨 (requires_action)")
            
            # submit_tool_outputs에 필요한 정보 수집
            tool_outputs = []
            
            # required_action.submit_tool_outputs.tool_calls 리스트 확인
            if run.required_action and run.required_action.submit_tool_outputs:
                for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                    if tool_call.function.name == "get_weather":
                        import json
                        args = json.loads(tool_call.function.arguments)
                        print(f" -> 함수 호출 감지: {tool_call.function.name}, 인자: {args}")
                        
                        # 함수 실행
                        result = get_weather(args["location"])
                        
                        # 결과 추가
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": result
                        })
            
            # 결과 제출
            if tool_outputs:
                print(" -> 함수 실행 결과 제출 중...")
                client.agents.runs.submit_tool_outputs(
                    thread_id=thread.id,
                    run_id=run.id,
                    tool_outputs=tool_outputs
                )

    if run.status == "failed":
        print(f"실행 실패: {run.last_error}")
    else:
        messages = client.agents.messages.list(thread_id=thread.id)
        messages_list = list(messages)
        if messages_list:
            print(f"에이전트 응답: {messages_list[0].content[0].text.value}")

    # 에이전트 정리
    client.agents.delete_agent(weather_agent.id)
    print("날씨 에이전트 삭제됨")

except Exception as e:
    traceback.print_exc()
    # 정리
    try:
        if 'weather_agent' in locals():
            client.agents.delete_agent(weather_agent.id)
    except:
        pass

In [50]:
# [고급 예제] 파일 검색 (File Search) / RAG 도구 사용
# 문서를 업로드하고, 해당 문서의 내용을 기반으로 답변하는 에이전트를 생성합니다.

try:
    print("--- 파일 검색(File Search) 테스트 시작 ---")
    
    # 필요한 모델 클래스 임포트 (딕셔너리 대신 객체 사용 필요)
    try:
        from azure.ai.projects.models import ToolResources, FileSearchToolResource
    except ImportError:
        from azure.ai.agents.models import ToolResources, FileSearchToolResource

    # 1. 검색할 가상 파일 생성
    file_name = "product_info.txt"
    with open(file_name, "w", encoding="utf-8") as f:
        f.write("Azure AI Foundry는 AI 앱 개발을 위한 통합 플랫폼입니다.\n")
        f.write("이 플랫폼은 모델 허브, 프롬프트 플로우, 그리고 다양한 보안 기능을 제공합니다.\n")
        f.write("특히 Agent 서비스를 통해 자율적인 AI 에이전트를 쉽게 구축할 수 있습니다.\n")
    
    # 2. 파일 업로드 (목적: assistants)
    uploaded_file = client.agents.files.upload(file_path=file_name, purpose="assistants")
    print(f"파일 업로드 완료: {uploaded_file.id}")

    # 3. 벡터 스토어(Vector Store) 생성 및 파일 연결
    vector_store = client.agents.vector_stores.create(name="my-knowledge-base")
    
    # 벡터 스토어에 파일 배치 업로드
    file_batch = client.agents.vector_store_file_batches.create_and_poll(
        vector_store_id=vector_store.id,
        file_ids=[uploaded_file.id]
    )
    print(f"벡터 스토어({vector_store.id})에 파일 연결 완료. 상태: {file_batch.status}")

    # 4. 파일 검색 도구 정의
    file_search_tool = FileSearchToolDefinition()

    # 5. 에이전트 생성
    search_agent = client.agents.create_agent(
        model=deployment_name,
        name="doc-search-bot",
        instructions="제공된 지식 기반(Knowledge Base)에서 정보를 찾아 답변하세요.",
        tools=[file_search_tool],
        tool_resources=ToolResources(
            file_search=FileSearchToolResource(
                vector_store_ids=[vector_store.id]
            )
        )
    )
    print(f"검색 에이전트 생성됨: {search_agent.id}")

    # 6. 질문하기
    thread = client.agents.threads.create()
    client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content="Azure AI Foundry가 무엇이고 어떤 기능을 제공해?"
    )

    # 7. 실행 (Polling 방식 사용)
    run = client.agents.runs.create(thread_id=thread.id, agent_id=search_agent.id)
    
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = client.agents.runs.get(thread_id=thread.id, run_id=run.id)
        print(f"실행 상태: {run.status}")
    
    if run.status == "completed":
        messages = client.agents.messages.list(thread_id=thread.id)
        messages_list = list(messages)
        if messages_list:
            # 주석(annotation)이 포함될 수 있으므로 텍스트만 추출
            response_text = messages_list[0].content[0].text.value
            print(f"에이전트 응답: {response_text}")
    else:
        print(f"실행 상태: {run.status}")
        if run.last_error:
            print(f"오류: {run.last_error}")

    # 정리 (에이전트, 벡터 스토어, 파일 삭제)
    print("리소스 정리 중...")
    client.agents.delete_agent(search_agent.id)
    client.agents.vector_stores.delete(vector_store.id)
    client.agents.files.delete(uploaded_file.id)
    import os
    if os.path.exists(file_name):
        os.remove(file_name)
    print("정리 완료")

except Exception as e:
    # 403 Forbidden 오류 처리
    if "403" in str(e) and "Authorization" in str(e):
        print("!!! 권한 오류 발생 !!!")
        print("Azure AI Project의 Managed Identity가 Storage Account에 접근할 권한이 없습니다.")
        print("'Storage Blob Data Contributor' 역할을 'infra/main.bicep'에 추가하고 'azd up'을 다시 실행해야 합니다.")
    
    traceback.print_exc()
    # 정리
    try:
        if 'search_agent' in locals(): client.agents.delete_agent(search_agent.id)
        if 'vector_store' in locals(): client.agents.vector_stores.delete(vector_store.id)
        if 'uploaded_file' in locals(): client.agents.files.delete(uploaded_file.id)
    except:
        pass

--- 파일 검색(File Search) 테스트 시작 ---
파일 업로드 완료: assistant-Pzgxeh5FsNupKwBj8MRWQ8
벡터 스토어(vs_NuSa0cRAZpLGdJrYZli1Eolk)에 파일 연결 완료. 상태: VectorStoreFileBatchStatus.COMPLETED
검색 에이전트 생성됨: asst_RtzzEWheAPPaIyUwmb3C7CYg
실행 상태: RunStatus.IN_PROGRESS
실행 상태: RunStatus.COMPLETED
에이전트 응답: Azure AI Foundry는 AI 애플리케이션 개발을 위한 통합 플랫폼입니다. 이 플랫폼은 여러 가지 기능을 제공하며, 주요 기능으로는 모델 허브, 프롬프트 플로우, 다양한 보안 기능이 포함됩니다. 특히, Azure AI Foundry는 Agent 서비스를 통해 자율적인 AI 에이전트를 쉽게 구축할 수 있도록 지원합니다【4:0†source】.
리소스 정리 중...
정리 완료
